# Unit V: Neural Networks (Deep Learning)
## Objective: Forecast Stock Prices using Long Short-Term Memory (LSTM) Networks.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout

%matplotlib inline

In [ ]:
# Load Data
df = pd.read_csv('../data/nifty50_2000_2025.csv', header=[0, 1], index_col=0, parse_dates=True)
df.columns = [col[0] for col in df.columns]

# Use only Closing Price
data = df.filter(['Close'])
dataset = data.values

# Scaling (LSTM requires 0-1 scale)
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(dataset)

print('Scaled Data Shape:', scaled_data.shape)

In [ ]:
# Create Training Data (Last 60 days to predict next day)
training_data_len = int(np.ceil(len(dataset) * .95))

train_data = scaled_data[0:training_data_len, :]

x_train = []
y_train = []

for i in range(60, len(train_data)):
    x_train.append(train_data[i-60:i, 0])
    y_train.append(train_data[i, 0])
    
x_train, y_train = np.array(x_train), np.array(y_train)

# Reshape for LSTM [Samples, Time Steps, Features]
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
print('X_train Shape:', x_train.shape)

In [ ]:
# Build LSTM Model
model = Sequential()
model.add(LSTM(50, return_sequences=True, input_shape=(x_train.shape[1], 1)))
model.add(LSTM(50, return_sequences=False))
model.add(Dense(25))
model.add(Dense(1))

model.compile(optimizer='adam', loss='mean_squared_error')
model.summary()

In [ ]:
# Train Model
history = model.fit(x_train, y_train, batch_size=1, epochs=1)

In [ ]:
# Create Test Data
test_data = scaled_data[training_data_len - 60: , :]

x_test = []
y_test = dataset[training_data_len:, :]

for i in range(60, len(test_data)):
    x_test.append(test_data[i-60:i, 0])
    
x_test = np.array(x_test)
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))

# Predictions
predictions = model.predict(x_test)
predictions = scaler.inverse_transform(predictions)

# RMSE
rmse = np.sqrt(np.mean(((predictions - y_test) ** 2)))
print('RMSE:', rmse)

In [ ]:
# Visualize Results
train = data[:training_data_len]
valid = data[training_data_len:]
valid['Predictions'] = predictions

plt.figure(figsize=(16, 8))
plt.title('LSTM Model Forecast')
plt.xlabel('Date')
plt.ylabel('Close Price (INR)')
plt.plot(train['Close'])
plt.plot(valid[['Close', 'Predictions']])
plt.legend(['Train', 'Val', 'Predictions'], loc='lower right')
plt.show()